In [314]:
import pandas as pd
import numpy as np


def answer_one():
    energy = pd.read_excel(
        'Energy Indicators.xls', skiprows=list(range(0, 18)) + list(range(245, 284)), usecols="C:F",  header=None,  names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
    pd.set_option('future.no_silent_downcasting', True)
    energy = energy.replace("...", np.nan)
    energy['Energy Supply'] = energy['Energy Supply'].copy() * 10**6
    energy['Country'] = energy['Country'].str.replace(r'\s*\(.*?\)', '', regex=True)
    energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)
    energy['Country'] = energy['Country'].replace(["Republic of Korea", "United States of America", "United Kingdom of Great Britain and Northern Ireland",
                                                "China, Hong Kong Special Administrative Region"], ["South Korea", "United States", "United Kingdom", "Hong Kong"])

    GDP = pd.read_csv('GDP.csv', skiprows=5, header=None,)
    GDP.columns = ["Country", "Country Code", "Indicator Name",
                "Indicator Code"]+[str(year) for year in range(1960, 2025)]
    GDP["Country"] = GDP["Country"].replace(
        ["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"], ["South Korea", "Iran", "Hong Kong"])
    GDP_years = GDP.copy()[["Country"] + [str(year) for year in range(2006, 2016)]]

    ScimEn = pd.read_excel('ScimEn.xlsx')
    merged_df = ScimEn.iloc[0:15].drop(columns=['Region']).merge(
        energy, on='Country').merge(GDP_years, on='Country')
    resulted_df = merged_df.set_index('Country')
    
    return (resulted_df)


TOP15 = answer_one()
print(TOP15)

                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     402188             400158    5077232   
United States          2     208042             203356    3540288   
India                  3      81853              79757     968523   
Japan                  4      58342              57735     717946   
United Kingdom         5      56288              54713    1095371   
Germany                6      50906              49773     777362   
Russian Federation     7      46186              45868     217996   
Canada                 8      41209              40390     915491   
Italy                  9      38700              36909     639473   
South Korea           10      38362              38046     618762   
Iran                  11      32080              31725     634135   
France                12      30810              30139     547122   
Spain                 13      2984

In [308]:
def answer_two(df):
    avgGDP = df.iloc[:, -10:].mean(axis=1, skipna=True)
    avgGDP = avgGDP.to_frame(name="Average GDP").sort_values(
        by="Average GDP", ascending=False)
    
    return avgGDP
    

avgGDP_for_country = answer_two(TOP15)
print(avgGDP_for_country)

                     Average GDP
Country                         
United States       1.572243e+13
China               6.927707e+12
Japan               5.239642e+12
Germany             3.590729e+12
United Kingdom      2.777505e+12
France              2.692000e+12
Italy               2.152983e+12
Brazil              1.988889e+12
Russian Federation  1.666746e+12
Canada              1.616359e+12
India               1.602352e+12
Spain               1.406644e+12
South Korea         1.221328e+12
Australia           1.207997e+12
Iran                4.567516e+11


In [309]:
def answer_three(df, Country):
    gdp_2006 = df.loc[Country, "2006"]
    gdp_2015 = df.loc[Country, "2015"]
    GDP_change = (gdp_2015 - gdp_2006)
    
    return f"{GDP_change:.6e}"
    
    
print(answer_three(TOP15, avgGDP_for_country.index[5]))

1.246219e+11


In [310]:
def answer_four(df):
    df["Self-Citation Ratio"] = df["Self-citations"] / df["Citations"]
    max_ratio_country = df["Self-Citation Ratio"].idxmax()
    max_ratio_value = df["Self-Citation Ratio"].max()
    result = (max_ratio_country, max_ratio_value)
    
    return result


print(answer_four(TOP15))

('China', 0.6916510807463594)


In [311]:
def answer_five(df):
    df["Estimated population"] = (df["Energy Supply"] / \
        df["Energy Supply per Capita"])
    global estimated_population_for_country
    estimated_population_for_country = df["Estimated population"]
    df = df.sort_values(by="Estimated population", ascending=False)
    
    return df.index[2]


print(answer_five(TOP15))

United States


In [312]:
def answer_six(df):
    df["Citable docs per capita"] = (
        df["Citable documents"] / estimated_population_for_country)
    correlation = df['Citable docs per capita'].corr(
        df['Energy Supply per Capita'], method='pearson')
    
    return correlation


print(answer_six(TOP15))

0.7019601723071187


In [313]:
def answer_seven(df):
    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'}
    df['Estimated population'] = estimated_population_for_country
    df['Continent'] = df.index.map(ContinentDict)
    grouped = df.groupby('Continent')
    size = grouped['Estimated population'].count()
    total_population = grouped['Estimated population'].sum()
    mean_population = grouped['Estimated population'].mean()
    std_population = grouped['Estimated population'].std()
    result = pd.DataFrame({
        'size': size,
        'sum': total_population,
        'mean': mean_population,
        'std': std_population
    })
    return result

print(answer_seven(TOP15))



               size               sum              mean           std
Continent                                                            
Asia              5   2898666386.6106   579733277.32212  6.790979e+08
Australia         1   23316017.316017   23316017.316017           NaN
Europe            6  457929667.216372   76321611.202729  3.464767e+07
North America     2   352855249.48025  176427624.740125  1.996696e+08
South America     1  205915254.237288  205915254.237288           NaN
